## 1. K-NN 알고리즘

### 모델 K 값 적용 시
1. 생존 여부를 분류할 때 영향을 주는 변수를 선택해서 
2. k(최근접을 몇 개까지 볼 것인지 지정)는 되도록 작은 수를 설정하고 홀수로 설정해서 분류분석을 수행합니다.
3. 데이터셋에서 생존자 클래스(생존자, 비생존자)의 데이터 수가  동일하다면 정확률로, 생존자 클래스의 데이터 수가 상이하다면 f1통계량으로 모델의 정확도를 판단한다.
4. 통상적으로 k=1일때 overfitting(학습 후 test에서 예측하지 못하는 경우) 발생할 가능성이 높습니다

#### 모델 적용전
- 변수를 0과 1로 채우는 정규화 과정(잡음제거)을 거침
- 중복열을 제거하여 다중공선성 문제해결

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')

In [3]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [4]:
#NaN 값 많은 deck(열) 삭제
#embarked와 embark_town 열(변수)는 의미가 동일하므로 embark_town 열 삭제
#del df['embark_town']
print(len(df.columns))
ndf = df.drop(['deck', 'embark_town'], axis=1)
print(len(ndf.columns))

15
13


In [5]:
#age 변수값이 NaN인 행을 삭제
print(len(ndf))
ndf.dropna(subset=['age'], how='any', axis=0, inplace=True) #how any일경우 하나라도 포함된다면 해당 행(axis =0) 삭제)
print(len(ndf))
print(ndf.describe(include='all'))

891
714
          survived      pclass   sex         age       sibsp       parch  \
count   714.000000  714.000000   714  714.000000  714.000000  714.000000   
unique         NaN         NaN     2         NaN         NaN         NaN   
top            NaN         NaN  male         NaN         NaN         NaN   
freq           NaN         NaN   453         NaN         NaN         NaN   
mean      0.406162    2.236695   NaN   29.699118    0.512605    0.431373   
std       0.491460    0.838250   NaN   14.526497    0.929783    0.853289   
min       0.000000    1.000000   NaN    0.420000    0.000000    0.000000   
25%       0.000000    1.000000   NaN   20.125000    0.000000    0.000000   
50%       0.000000    2.000000   NaN   28.000000    0.000000    0.000000   
75%       1.000000    3.000000   NaN   38.000000    1.000000    1.000000   
max       1.000000    3.000000   NaN   80.000000    5.000000    6.000000   

              fare embarked  class  who adult_male alive alone  
count   714.00

In [6]:
print(len(ndf['embarked'].isnull()))

714


In [7]:
#embark 열 NaN값을 승선도시 중 가장 많이 출연한 데이터 값으로 치환
#ndf['embarked'].value_counts(dropna=True).idxmax()
most_freq = ndf['embarked'].value_counts(dropna=True).idxmax()
print(most_freq)

ndf['embarked'].fillna(most_freq, inplace=True)
print(ndf.describe(include='all'))

S
          survived      pclass   sex         age       sibsp       parch  \
count   714.000000  714.000000   714  714.000000  714.000000  714.000000   
unique         NaN         NaN     2         NaN         NaN         NaN   
top            NaN         NaN  male         NaN         NaN         NaN   
freq           NaN         NaN   453         NaN         NaN         NaN   
mean      0.406162    2.236695   NaN   29.699118    0.512605    0.431373   
std       0.491460    0.838250   NaN   14.526497    0.929783    0.853289   
min       0.000000    1.000000   NaN    0.420000    0.000000    0.000000   
25%       0.000000    1.000000   NaN   20.125000    0.000000    0.000000   
50%       0.000000    2.000000   NaN   28.000000    0.000000    0.000000   
75%       1.000000    3.000000   NaN   38.000000    1.000000    1.000000   
max       1.000000    3.000000   NaN   80.000000    5.000000    6.000000   

              fare embarked  class  who adult_male alive alone  
count   714.000000  

In [8]:

# 범주형(문자) 데이터를 숫자로 변환 해주어야 함
# survived, pclass, sex, age, sibsp, parch, embarked

# 범주형 데이터를 모델이 인식할 수 있는 숫자로 변환 : one-hot encoding
onehot_sex = pd.get_dummies(ndf['sex'])
print(onehot_sex)

ndf = pd.concat([ndf, onehot_sex], axis=1)
print(ndf)



     female  male
0         0     1
1         1     0
2         1     0
3         1     0
4         0     1
..      ...   ...
885       1     0
886       0     1
887       1     0
889       0     1
890       0     1

[714 rows x 2 columns]
     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
885         0       3  female  39.0      0      5  29.1250        Q   Third   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0

In [24]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town') # embarked 의 컬럼을 각 변수에 따라 town_$로 지정한다.
print(onehot_embarked)
ndf = pd.concat([ndf, onehot_embarked], axis=1) # ndf에 onehot_embarked 결합한다.
print(ndf)


     town_C  town_Q  town_S
0         0       0       1
1         1       0       0
2         0       0       1
3         0       0       1
4         0       0       1
..      ...     ...     ...
885       0       1       0
886       0       0       1
887       0       0       1
889       1       0       0
890       0       1       0

[714 rows x 3 columns]
     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
885         0       3  female  39.0      0      5  29.1250        Q   Third   
886     

In [10]:
# 분류분석에 사용할 변수 선택

X = ndf[['pclass','female', 'male', 'age', 'sibsp', 'parch', 'town_C', 'town_Q', 'town_S']]
Y = ndf['survived']

In [11]:
# K-NN 분류 분석을 수행하려면 설정변수를 정규화 해주어야 함(평균 0 , 표준편차 1)
from sklearn import preprocessing
print(X)
X = preprocessing.StandardScaler().fit(X).transform(X)
print(X)
print(X.shape)

     pclass  female  male   age  sibsp  parch  town_C  town_Q  town_S
0         3       0     1  22.0      1      0       0       0       1
1         1       1     0  38.0      1      0       1       0       0
2         3       1     0  26.0      0      0       0       0       1
3         1       1     0  35.0      1      0       0       0       1
4         3       0     1  35.0      0      0       0       0       1
..      ...     ...   ...   ...    ...    ...     ...     ...     ...
885       3       1     0  39.0      0      5       0       1       0
886       2       0     1  27.0      0      0       0       0       1
887       1       1     0  19.0      0      0       0       0       1
889       1       0     1  26.0      0      0       1       0       0
890       3       0     1  32.0      0      0       0       1       0

[714 rows x 9 columns]
[[ 0.91123237 -0.75905134  0.75905134 ... -0.47180795 -0.20203051
   0.53307848]
 [-1.47636364  1.31743394 -1.31743394 ...  2.11950647 -

In [13]:
# train data와 test data 비율 7:3으로 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)
print(X_train.shape)
print(X_test.shape)


(499, 9)
(215, 9)


In [57]:
# KNN 분류 분석으로 모델 생성
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [58]:
# 학습 데이터로부터 생성된 모델로 예측값 생성
y_predict = knn.predict(X_test)


In [49]:
print(list(y_predict[:10]))
print(list(Y_test[:10]))

[0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0]


In [59]:
# 모델 검증
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(Y_test, y_predict)
print(knn_matrix) # 혼동매트릭스
#print(knn_matrix.shape)

knn_report = metrics.classification_report(Y_test, y_predict)
print(knn_report)
#precision 값이 0과 1이 0.81,0.80 

[[113  12]
 [ 30  60]]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       125
           1       0.83      0.67      0.74        90

    accuracy                           0.80       215
   macro avg       0.81      0.79      0.79       215
weighted avg       0.81      0.80      0.80       215

